## Image classification with Convolutional Neural Networks

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch, cv2

In [3]:
# This file contains all the main external libs we'll use
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [4]:
PATH = "data/dogscats/"
sz=224
arch=vgg16
bs=64

In [5]:
# Uncomment the below if you need to reset your precomputed activations
# !rm -rf {PATH}tmp

In [6]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz))

In [7]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /home/ec2-user/.torch/models/vgg16_bn-6c64b313.pth
100%|██████████| 553507836/553507836 [00:04<00:00, 112933267.31it/s]


100%|██████████| 32/32 [00:30<00:00,  1.07it/s]


In [8]:
learn.fit(0.01, 3, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                     
    0      0.055043   0.028882   0.990234  
    1      0.053605   0.02688    0.990723                     
    2      0.052409   0.024518   0.991699                     



[0.024518285, 0.99169921875]

In [9]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)

In [10]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs, num_workers=4)
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [11]:
learn.fit(1e-2, 2)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                     
    0      0.053914   0.031593   0.98584   
    1      0.046005   0.031041   0.987305                     



[0.031041332, 0.9873046875]

In [12]:
learn.precompute=False

In [13]:
learn.fit(1e-2, 1, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                     
    0      0.052846   0.031657   0.987305  



[0.031657223, 0.9873046875]

In [14]:
learn.unfreeze()

In [15]:
lr=np.array([1e-4,1e-3,1e-2])

In [16]:
learn.fit(lr, 1, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                     
    0      0.062483   0.028339   0.988281  



[0.02833863, 0.98828125]

In [ ]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                     
    0      0.049969   0.025686   0.987793  
    1      0.041388   0.022591   0.989746                     
    2      0.03521    0.022123   0.98877                      
 45%|████▌     | 163/360 [09:48<11:50,  3.61s/it, loss=0.0367]

In [ ]:
learn.fit(lr, 3, cycle_len=3)

In [ ]:
log_preds,y = learn.TTA()
accuracy(log_preds,y)

In [ ]:
print("log_preds:", type(log_preds), log_preds.shape)
print("y:", type(y), y.shape)
probs = np.mean(np.exp(log_preds),0)
print ("probs:", type(probs), probs.shape)